In [1]:
import sys, os
import torch
import librosa
import numpy as np
import pandas as pd
from torch import Tensor
from scipy.io import wavfile
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import scipy.io.wavfile
import wave
import struct
# import torchaudio


In [2]:
class RawNetDataset(Dataset):
    def __init__(self, transform=None, mode="train",files_dir=None, base_dir="",csv_file_dir="",nb_time=59049):
        self.base_dir = base_dir
        self.mode = mode
        self.transform = transform
        self.csv_file_dir = csv_file_dir
        self.files_dir = files_dir
        csv_file = pd.read_csv(csv_file_dir,sep="\t")
        self.nb_time = nb_time
        self.classes = np.array([i for i, cls_name in enumerate(csv_file["VoxCeleb1 ID"].unique())])

    def __len__(self):
        return self.classes.shape[0] 
    
    def __getitem__(self, idx):
        filename = self.files_dir[idx]
        X, sample_rate = librosa.load(self.base_dir+filename)
       
       
        
        label = self.classes[idx]
        X1 = torch.from_numpy(X)
        
#         print(type(X))
#         print(type(X1))
        X1.resize_((1,X1.shape[0]))
# #         print(X1[:,1:])
#         print(X.shape)
        print(X1.shape)
       
        
        self._pre_emphasis(X1)
        nb_time = X1.shape[0]
        print(nb_time)
        print(self.nb_time)
#         if nb_time > self.nb_time:
#             start_idx = np.random.randint(low = 0, high = nb_time - self.nb_time)
#             X1 = X1[:, start_idx:start_idx+self.nb_time]
#             print("taille supérieure")
#         elif nb_time < self.nb_time:
#             nb_dup = int(self.nb_time / nb_time) + 1
#             X1 = np.tile(X1, (1, nb_dup))[:, :self.nb_time]
#             print("taille inférieure")
#         else:
#             X1 = X1
#             print("taille égale")
#             print(X1)
#             print(label)
        print()
        return X1, label

    def _pre_emphasis(self, x):
        '''
        Pre-emphasis for single channel input
        '''
        return np.asarray(x[:,1:] - 0.97 * x[:, :-1], dtype=np.float32) 



In [10]:
base_dir = 'data/wav'
list_IDs = ['/id10009/7hpSiT9_gCE/00001.wav',
        '/id10009/aFttHpeaXaQ/00001.wav',
        '/id10009/AtavJVP4bCk/00001.wav',
        '/id10009/AtavJVP4bCk/00002.wav',
        '/id10009/AtavJVP4bCk/00003.wav',
        '/id10009/AtavJVP4bCk/00004.wav',
        '/id10009/AtavJVP4bCk/00005.wav',
        '/id10009/AtavJVP4bCk/00006.wav',
        '/id10009/AtavJVP4bCk/00007.wav',
        '/id10009/AtavJVP4bCk/00008.wav',
        ]
# list_IDs = ['/id10009/7hpSiT9_gCE/00001.wav',
#         '/id10009/aFttHpeaXaQ/00001.wav',
#         '/id10009/AtavJVP4bCk/00001.wav',
#         '/id10009/AtavJVP4bCk/00002.wav'
#         ]
csv_file_dir = 'data/target/vox1_meta.csv'

In [28]:
def get_utt_list(src_dir):
    l_utt = []
    for r, ds, fs in os.walk(src_dir):
        r = r.replace('\\', '/')   
        base = '/'.join(r.split('/')[-2:])+'/'
        for f in fs:
            l_utt.append(base+f[:-4]+'.wav')
    return l_utt


In [35]:
src_dir = 'data/wav'
csv_file_dir2 = 'data/target/vox1_meta.csv'
l = get_utt_list(src_dir)
print(l)

['data/wav/._id1.wav', 'data/wav/._id1.wav', 'data/wav/._id1.wav', 'data/wav/._id1.wav', 'data/wav/._id1.wav', 'wav/id10016/._pdTdefd.wav', 'wav/id10016/._ywRolvE.wav', 'wav/id10016/._7OacMAs.wav', 'wav/id10016/._WfkA4lU.wav', 'wav/id10016/._l5sY-WW.wav', 'wav/id10016/._5xGJYiN.wav', 'wav/id10016/._uM5GCGO.wav', 'wav/id10016/._37mbJ9b.wav', 'wav/id10016/._LSeSZiv.wav', 'wav/id10016/._MPkuYnD.wav', 'wav/id10016/._WwguhI5.wav', 'wav/id10016/._SCFgvk0.wav', 'wav/id10016/._hgB5ziA.wav', 'wav/id10016/._bIi7FnN.wav', 'wav/id10016/._MMTrblY.wav', 'wav/id10016/._Zxab80I.wav', 'wav/id10016/._FDXUYIg.wav', 'wav/id10016/._ocnz4PP.wav', 'wav/id10016/._o524HaR.wav', 'wav/id10016/.DS_S.wav', 'wav/id10016/._RREb7K-.wav', 'wav/id10016/._vpnw2jQ.wav', 'wav/id10016/._PnSNt2F.wav', 'wav/id10016/._.DS_S.wav', 'wav/id10016/._2wHaOQU.wav', 'wav/id10016/._dOnuu_9.wav', 'wav/id10016/._wg6IOBM.wav', 'wav/id10016/._j0CaA8L.wav', 'wav/id10016/._fb31Etx.wav', 'wav/id10016/._CsU9a10.wav', 'wav/id10016/._7FOcBtZ.wa

In [36]:
dataset = RawNetDataset(files_dir=list_IDs,base_dir=src_dir,csv_file_dir=csv_file_dir2)
data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=1,
                                          shuffle=True)

In [37]:
dataset.__getitem__(5)

torch.Size([1, 205508])
1
59049



(tensor([[ 0.0133,  0.0103, -0.0033,  ..., -0.0066, -0.0085,  0.0000]]), 5)

In [38]:
print(len(dataset))

1251
